## Six datasets for the 10x genomic breast cancer

Siyu

2021-09-23

In [ ]:
## mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip3 install anndata
#!pip3 install scanpy
#!pip3 install git+https://github.com/jacoblevine/phenograph.git
#!pip3 install leidenalg
#!pip install matplotlib==3.3.4
#!pip install py_pcha
#!pip install spams

In [ ]:
import os
import sys
import gzip
import numpy as np
import scipy.stats as stats
import pandas as pd
import pickle
import anndata
import json
import scanpy as sc
import scipy
import seaborn as sns
import phenograph 
import collections
import torch
from scipy.io import mmread,mminfo
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from skimage import io
from py_pcha import PCHA
sys.path.append('/content/drive/MyDrive/SpatialModelProject/VAE')

In [ ]:
import torch
torch.manual_seed(0)
#plt.rcParams["figure.figsize"] = (8, 8)

### download datasets

dataset1: BC_A1
https://support.10xgenomics.com/spatial-gene-expression/datasets/1.1.0/V1_Breast_Cancer_Block_A_Section_1

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A1/ https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A1/ https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_analysis.tar.gz
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A1/ https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_spatial.tar.gz
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A1/ https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_metrics_summary.csv

dataset2: BC_A2
https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Breast_Cancer_Block_A_Section_2

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A2 https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A2 https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.tar.gz
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A2 https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_analysis.tar.gz
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A2 https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_spatial.tar.gz
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A2 https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_metrics_summary.csv
!tar zxvf /content/drive/MyDrive/SpatialModelProject/data/BC_A2/wget/V1_Breast_Cancer_Block_A_Section_2_spatial.tar.gz -C /content/drive/MyDrive/SpatialModelProject/data/BC_A2

datasets3: BC_A3 https://support.10xgenomics.com/spatial-gene-expression/datasets/1.1.0/V1_Breast_Cancer_Block_A_Section_2

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A3 https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A3 https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_spatial.tar.gz
!tar zxvf /content/drive/MyDrive/SpatialModelProject/data/BC_A3/V1_Breast_Cancer_Block_A_Section_2_spatial.tar.gz -C /content/drive/MyDrive/SpatialModelProject/data/BC_A3

datasets4: BC_A4 https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Breast_Cancer_Block_A_Section_2

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A4/ https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A4/ https://cf.10xgenomics.com/samples/spatial-exp/1.0.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_spatial.tar.gz
!tar zxvf /content/drive/MyDrive/SpatialModelProject/data/BC_A4/V1_Breast_Cancer_Block_A_Section_1_spatial.tar.gz -C /content/drive/MyDrive/SpatialModelProject/data/BC_A4

datasets5: BC_A5 https://support.10xgenomics.com/spatial-gene-expression/datasets/1.2.0/Parent_Visium_Human_BreastCancer

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A5/ https://cf.10xgenomics.com/samples/spatial-exp/1.2.0/Parent_Visium_Human_BreastCancer/Parent_Visium_Human_BreastCancer_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A5/ https://cf.10xgenomics.com/samples/spatial-exp/1.2.0/Parent_Visium_Human_BreastCancer/Parent_Visium_Human_BreastCancer_spatial.tar.gz
!tar zxvf /content/drive/MyDrive/SpatialModelProject/data/BC_A5/Parent_Visium_Human_BreastCancer_spatial.tar.gz -C /content/drive/MyDrive/SpatialModelProject/data/BC_A5

datasets: BC_A6 https://support.10xgenomics.com/spatial-gene-expression/datasets/1.2.0/Targeted_Visium_Human_BreastCancer_Immunology

In [ ]:
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A6/ https://cf.10xgenomics.com/samples/spatial-exp/1.2.0/Targeted_Visium_Human_BreastCancer_Immunology/Targeted_Visium_Human_BreastCancer_Immunology_filtered_feature_bc_matrix.h5
!wget -P /content/drive/MyDrive/SpatialModelProject/data/BC_A6/ https://cf.10xgenomics.com/samples/spatial-exp/1.2.0/Targeted_Visium_Human_BreastCancer_Immunology/Targeted_Visium_Human_BreastCancer_Immunology_spatial.tar.gz

!tar zxvf /content/drive/MyDrive/SpatialModelProject/data/BC_A6/Targeted_Visium_Human_BreastCancer_Immunology_spatial.tar.gz -C /content/drive/MyDrive/SpatialModelProject/data/BC_A5